In [1]:
import numpy as np

In [2]:
def celerite_factor(A, U, V, P):
    N, J = U.shape
    S = np.zeros((J, J))
    W = np.empty_like(U)
    D = np.empty_like(A)
    
    D[0] = A[0]
    W[0] = V[0] / D[0]
    for n in range(1, N):
        S = np.dot(
            np.diag(P[n-1]),
            np.dot(
                S + D[n-1]*np.dot(W[n-1:n].T, W[n-1:n]),
                np.diag(P[n-1])
            )
        )
        
        D[n] = A[n] - np.dot(U[n], np.dot(S, U[n]))
        
        W[n] = (V[n] - np.dot(U[n], S)) / D[n]
    
    return D, W, S

# A, U, V, P,
def celerite_factor_grad(U, P, D, W, S, bD, bW, bS):
    N, J = U.shape
    bA = np.zeros_like(D)
    bU = np.zeros_like(U)
    bV = np.zeros_like(U)
    bP = np.zeros_like(P)

    S_ = np.array(S)
    bD_ = bD[-1]
    bW_ = np.array(bW[-1:]) / D[-1]
    bS_ = np.array(bS)
    
    for n in range(N-1, 0, -1):
        # Step 6
        bD_ -= float(np.dot(W[n:n+1], bW_.T))
        bV[n:n+1] += bW_
        bU[n:n+1] -= np.dot(bW_, S_)
        bS_ -= np.dot(U[n:n+1].T, bW_)
        
        # Step 5
        bA[n] += bD_
        bU[n] -= 2*bD_*np.dot(U[n], S_)
        bS_ -= bD_*np.dot(U[n:n+1].T, U[n:n+1])

        # Step 4
        S_ = np.dot(S_, np.diag(1./P[n-1]))
        bP[n-1] = np.diag(np.dot(bS_, S_) + np.dot(S_.T, bS_))
        
        # Step 3
        bS_ = np.dot(np.diag(P[n-1]), np.dot(bS_, np.diag(P[n-1])))
        bD_ = bD[n-1] + float(np.dot(W[n-1:n], np.dot(bS_, W[n-1:n].T)))
        bW_ = (bW[n-1:n] + D[n-1]*np.dot(W[n-1:n], (bS_ + bS_.T))) / D[n-1]
        
        # Downdate S
        S_ = np.dot(np.diag(1.0/P[n-1]), S_) - D[n-1]*np.dot(W[n-1:n].T, W[n-1:n])
        
    bV[:1] += bW_
    bD_ -= float(np.dot(bW_, W[:1].T))
    bA[0] = bD_

    return bA, bU, bV, bP

In [3]:
N = 100
J_real = 2
J_comp = 3

np.random.seed(123)
x = np.sort(np.random.uniform(0, 10, N))
diag = np.random.uniform(0.5, 1.5, N)

a_real = np.random.rand(J_real)
c_real = np.random.rand(J_real)

a_comp = np.random.rand(J_comp)
b_comp = np.random.rand(J_comp)
c_comp = np.random.rand(J_comp)
d_comp = np.random.rand(J_comp)

A = diag + np.sum(a_real) + np.sum(a_comp)

U = np.concatenate((
    a_real + np.zeros((N, J_real)),
    a_comp[None, :] * np.cos(d_comp[None, :] * x[:, None])
    + b_comp[None, :] * np.sin(d_comp[None, :] * x[:, None]),
    a_comp[None, :] * np.sin(d_comp[None, :] * x[:, None])
    - b_comp[None, :] * np.cos(d_comp[None, :] * x[:, None]),
), axis=1)

V = np.concatenate((
    np.ones((N, J_real)),
    np.cos(d_comp[None, :] * x[:, None]),
    np.sin(d_comp[None, :] * x[:, None]),
), axis=1)

dx = x[1:] - x[:-1]
P = np.concatenate((
    np.exp(-c_real[None, :] * dx[:, None]),
    np.exp(-c_comp[None, :] * dx[:, None]),
    np.exp(-c_comp[None, :] * dx[:, None]),
), axis=1)

D, W, S = celerite_factor(A, U, V, P)

# bD = 1.0 / D
bD = np.ones_like(D)
bW = np.ones_like(W)
bS = np.ones_like(S)

bA, bU, bV, bP = celerite_factor_grad(U, P, D, W, S, bD, bW, bS)

In [4]:
bA, bU, bV, bP

(array([ 1.04268617,  0.69217753,  0.2202681 ,  0.59764672,  0.5187299 ,
         0.60524864,  0.73702773,  0.51263661,  0.90967098,  0.63158887,
         0.44396611,  0.60276161,  0.32666822,  0.70982217,  1.0964064 ,
         0.77776408,  0.85307548,  0.74032135,  0.23589992,  1.01251531,
         0.97411722,  1.196664  ,  0.90671668,  0.96181174,  0.97014226,
         0.94863725,  0.84863436,  1.10497903,  0.99422258,  0.92552986,
         1.00708436,  0.95870093,  0.94895756,  1.11697603,  1.04459934,
         1.32078494,  1.01991699,  0.97681643,  1.11275884,  1.04945973,
         1.21990141,  1.08850817,  1.01017472,  1.02704002,  1.01985374,
         1.0062342 ,  1.20039221,  1.43647174,  1.10877416,  1.08413054,
         1.28978838,  1.08717872,  1.26177481,  1.1391465 ,  1.11358722,
         1.34425039,  1.08005458,  1.04304085,  1.11091139,  1.21733094,
         1.19483668,  1.06664976,  1.10298446,  1.34406053,  1.07073629,
         1.08734418,  1.13567539,  1.03345205,  1.0

In [5]:
eps = 1e-8
ind = 0
A[ind] += eps
D_p, W_p, S_p = celerite_factor(A, U, V, P)
ld_p = np.sum(D_p) + np.sum(W_p) + np.sum(S_p)
A[ind] -= 2*eps
D_m, W_m, S_m = celerite_factor(A, U, V, P)
ld_m = np.sum(D_m) + np.sum(W_m) + np.sum(S_m)
A[ind] += eps
g = 0.5 * (ld_p - ld_m) / eps

In [6]:
g, bA[ind], g - bA[ind]

(1.042685937591159, 1.042686170522706, -2.329315469395965e-07)

In [7]:
bA

array([ 1.04268617,  0.69217753,  0.2202681 ,  0.59764672,  0.5187299 ,
        0.60524864,  0.73702773,  0.51263661,  0.90967098,  0.63158887,
        0.44396611,  0.60276161,  0.32666822,  0.70982217,  1.0964064 ,
        0.77776408,  0.85307548,  0.74032135,  0.23589992,  1.01251531,
        0.97411722,  1.196664  ,  0.90671668,  0.96181174,  0.97014226,
        0.94863725,  0.84863436,  1.10497903,  0.99422258,  0.92552986,
        1.00708436,  0.95870093,  0.94895756,  1.11697603,  1.04459934,
        1.32078494,  1.01991699,  0.97681643,  1.11275884,  1.04945973,
        1.21990141,  1.08850817,  1.01017472,  1.02704002,  1.01985374,
        1.0062342 ,  1.20039221,  1.43647174,  1.10877416,  1.08413054,
        1.28978838,  1.08717872,  1.26177481,  1.1391465 ,  1.11358722,
        1.34425039,  1.08005458,  1.04304085,  1.11091139,  1.21733094,
        1.19483668,  1.06664976,  1.10298446,  1.34406053,  1.07073629,
        1.08734418,  1.13567539,  1.03345205,  1.03260656,  1.02

In [17]:
U *= np.concatenate((
    np.exp(-c_real[None, :] * x[:, None]),
    np.exp(-c_comp[None, :] * x[:, None]),
    np.exp(-c_comp[None, :] * x[:, None]),
), axis=1)
V *= np.concatenate((
    np.exp(c_real[None, :] * x[:, None]),
    np.exp(c_comp[None, :] * x[:, None]),
    np.exp(c_comp[None, :] * x[:, None]),
), axis=1)
W *= np.concatenate((
    np.exp(c_real[None, :] * x[:, None]),
    np.exp(c_comp[None, :] * x[:, None]),
    np.exp(c_comp[None, :] * x[:, None]),
), axis=1)

L = np.tril(np.dot(U, W.T), -1) + np.eye(N)
K2 = np.dot(L, np.dot(np.diag(D), L.T))

K1 = np.tril(np.dot(U, V.T), -1) + np.triu(np.dot(V, U.T), 1) + np.diag(A)
print(K1 - K2)

[[  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.11022302e-16
    0.00000000e+00  -2.77555756e-17  -1.38777878e-17  -1.38777878e-17
   -1.38777878e-17   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00  -2.22044605e-16   0.00000000e+00
   -5.55111512e-17  -1.11022302e-16  -4.16333634e-17   1.38777878e-17
    0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00  -2.22044605e-16  -4.44089210e-16  -1.11022302e-16
   -1.11022302e-16  -5.55111512e-17  -1.38777878e-17   0.00000000e+00
    0.00000000e+00   2.77555756e-17]
 [  1.11022302e-16  -1.11022302e-16  -1.11022302e-16  -2.22044605e-16
   -1.11022302e-16   1.11022302e-16   1.11022302e-16   0.00000000e+00
    2.77555756e-17  -4.16333634e-17]
 [  0.00000000e+00  -5.55111512e-17  -1.11022302e-16   0.00000000e+00
    0.00000000e+00   0.00000000e+00   2.22044605e-16   0.00000000e+00
    0.00000000e+00   1.11022302e-16]
 [ -2.77555756e-17  -1.11022302e-16  -5.55111512e-17   1.11022302e-16
    0.00000000e+00   2.22044605e-16   2.22044

In [351]:
np.random.seed(123)
x = np.sort(np.random.uniform(0, 10, N))
diag = np.random.uniform(0.5, 1.5, N)

a_real = np.random.rand(J_real)
c_real = np.random.rand(J_real)

a_comp = np.random.rand(J_comp)
b_comp = np.random.rand(J_comp)
c_comp = np.random.rand(J_comp)
d_comp = np.random.rand(J_comp)

A = diag + np.sum(a_real) + np.sum(a_comp)

U = np.concatenate((
    a_real + np.zeros((N, J_real)),
    a_comp[None, :] * np.cos(d_comp[None, :] * x[:, None])
    + b_comp[None, :] * np.sin(d_comp[None, :] * x[:, None]),
    a_comp[None, :] * np.sin(d_comp[None, :] * x[:, None])
    - b_comp[None, :] * np.cos(d_comp[None, :] * x[:, None]),
), axis=1)

V = np.concatenate((
    np.ones((N, J_real)),
    np.cos(d_comp[None, :] * x[:, None]),
    np.sin(d_comp[None, :] * x[:, None]),
), axis=1)

dx = x[1:] - x[:-1]
P = np.concatenate((
    np.exp(-c_real[None, :] * dx[:, None]),
    np.exp(-c_comp[None, :] * dx[:, None]),
    np.exp(-c_comp[None, :] * dx[:, None]),
), axis=1)

In [355]:
K = np.diag(diag)
tau = np.abs(x[:, None] - x[None, :])
for j in range(J_real):
    K += a_real[j] * np.exp(-c_real[j] * tau)
for j in range(J_comp):
    K += a_comp[j] * np.exp(-c_comp[j] * tau) * np.cos(d_comp[j]*tau)
    K += b_comp[j] * np.exp(-c_comp[j] * tau) * np.sin(d_comp[j]*tau)

In [356]:
K

array([[ 3.06834523,  1.88507227,  1.76777237, ..., -0.00538622,
        -0.00524485, -0.00495898],
       [ 1.88507227,  3.22184163,  1.96120209, ..., -0.00620869,
        -0.00606396, -0.00576939],
       [ 1.76777237,  1.96120209,  2.66112556, ..., -0.00669498,
        -0.00655005, -0.00625377],
       ..., 
       [-0.00538622, -0.00620869, -0.00669498, ...,  3.29999773,
         2.02913994,  1.97065727],
       [-0.00524485, -0.00606396, -0.00655005, ...,  2.02913994,
         3.02813008,  2.00009618],
       [-0.00495898, -0.00576939, -0.00625377, ...,  1.97065727,
         2.00009618,  2.67697143]])

In [371]:
Up = np.array(U)
Vp = np.array(V)
Vp[:-1] *= P

np.diag(A) + np.tril(np.dot(Up, Vp.T), -1) + np.triu(np.dot(Vp, Up.T), 1) - K

array([[  0.00000000e+00,   0.00000000e+00,   1.99080873e-01, ...,
          3.99638741e-01,   3.83398651e-01,   3.48220417e-01],
       [  0.00000000e+00,   4.44089210e-16,   2.22044605e-16, ...,
          5.45015406e-01,   5.31792658e-01,   5.02128525e-01],
       [  1.99080873e-01,   2.22044605e-16,   4.44089210e-16, ...,
          5.36805173e-01,   5.27150124e-01,   5.04780706e-01],
       ..., 
       [  3.99638741e-01,   5.45015406e-01,   5.36805173e-01, ...,
          4.44089210e-16,  -4.44089210e-16,   1.33116134e-01],
       [  3.83398651e-01,   5.31792658e-01,   5.27150124e-01, ...,
         -4.44089210e-16,   4.44089210e-16,   4.44089210e-16],
       [  3.48220417e-01,   5.02128525e-01,   5.04780706e-01, ...,
          1.33116134e-01,   4.44089210e-16,   4.44089210e-16]])